In [1]:
import pandas as pd
import datetime
import csv

tmp_data_2017=pd.read_csv("E:\\5001\\KAGGLE\\2017_data.csv")
tmp_data_2018=pd.read_csv("E:\\5001\\KAGGLE\\2018_data.csv")
#print(tmp_data_2017.head())

D:\Anaconda\Ana\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (20,32,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
D:\Anaconda\Ana\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (18,21,25,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
def dt_transfermation(dt):
    dt["datetime_1"]=dt["DATE"].apply(lambda x:datetime.datetime.strptime(x[:13],"%Y-%m-%dT%H"))
    return dt
tmp_data_2017=dt_transfermation(tmp_data_2017)
tmp_data_2018=dt_transfermation(tmp_data_2018)
#print(tmp_data_2017.head())

In [3]:
from pytz import utc  
from pytz import timezone 

def timezone_transfermation(dt):
    cst_tz = timezone('Asia/Shanghai')  
    utc_tz = timezone('UTC') 
    dt["datetime"]=dt["datetime_1"].apply(lambda x:x.replace(tzinfo=utc_tz).astimezone(cst_tz))
    return dt
    
def temperature_transfermation(dt):
    dt["Temp"]=dt["TMP"].apply(lambda x:float(x[2:5]))
    return dt
    
tmp_data_2017=timezone_transfermation(tmp_data_2017)
tmp_data_2018=timezone_transfermation(tmp_data_2018)
tmp_data_2017=temperature_transfermation(tmp_data_2017)
tmp_data_2018=temperature_transfermation(tmp_data_2018)
tmp_data_2017=tmp_data_2017[["datetime","Temp"]]
tmp_data_2018=tmp_data_2018[["datetime","Temp"]]

In [4]:
train_data=pd.read_csv("E:\\5001\\KAGGLE\\train.csv")
test_data=pd.read_csv("E:\\5001\\KAGGLE\\test.csv")

In [5]:
train_dataset=train_data.copy()
test_dataset=test_data.copy()
    
def datetime_transfermation(dataset):
    dataset['datetime']=dataset['date'].apply(lambda x:datetime.datetime.strptime(x,"%d/%m/%Y %H:00"))
    return dataset
    
def get_day_month_year(dataset):
    dataset['month']=dataset['datetime'].apply(lambda x:int(x.month))
    dataset['day']=dataset['datetime'].apply(lambda x:int(x.day))
    dataset['year']=dataset['datetime'].apply(lambda x:int(x.year))
    return dataset
    
def get_hour(dataset):
    dataset['hour']=dataset['datetime'].apply(lambda x:int(x.hour))
    return dataset

def get_hour_feature(dataset,feature_select=True):
    if feature_select:
        def get_feature(x):
            time_range=[[0,1,2,3],[4,5,6,7],[8,9,10],[11,12],[13,14,15,16],[17,18,19],[20,21,22,23]]
            for i in range(len(time_range)):
                if x in time_range[i]:
                    return chr(i+97)
        dataset['hour_t']=dataset['hour'].apply(get_feature)
    else:
        dataset['hour_t']=dataset['hour'].apply(lambda x: chr(x+97))
    return dataset

def get_Weekday(dataset):
    dataset['Sun']=dataset['datetime'].apply(lambda x: 1 if x.weekday()==6 else 0) 
    dataset['Sat']=dataset['datetime'].apply(lambda x: 1 if x.weekday()==5 else 0)
    dataset['Fri']=dataset['datetime'].apply(lambda x: 1 if x.weekday()==4 else 0)
    dataset['Thu']=dataset['datetime'].apply(lambda x: 1 if x.weekday()==3 else 0) 
    dataset['Wed']=dataset['datetime'].apply(lambda x: 1 if x.weekday()==2 else 0)
    dataset['Tue']=dataset['datetime'].apply(lambda x: 1 if x.weekday()==1 else 0)
    dataset['Mon']=dataset['datetime'].apply(lambda x: 1 if x.weekday()==0 else 0)
    return dataset

def get_public_holiday(dataset):
    holiday_list= ['2017-01-02','2017-01-28','2017-01-30','2017-01-31','2017-04-04','2017-04-14',
           '2017-04-15','2017-04-17','2017-05-01','2017-05-03','2017-05-30','2017-07-01',
           '2017-10-02','2017-10-05','2017-10-28','2017-12-25','2017-12-26','2018-01-01',
           '2018-02-16','2018-02-17','2018-02-19','2018-03-30','2018-03-31','2018-04-02',
           '2018-04-05','2018-05-01','2018-05-22','2018-06-18','2018-07-02','2018-09-25',
           '2018-10-01','2018-10-17','2018-12-25','2018-12-26']
    dataset['holiday']=dataset['datetime'].apply(lambda x:1 if x.strftime('%Y-%m-%d') in holiday_list else 0)
    return dataset
    
def dataset_transformation_operations(train_dataset): 
    train_dataset=datetime_transfermation(train_dataset)
    train_dataset=get_day_month_year(train_dataset)
    train_dataset=get_hour(train_dataset)
    train_dataset=get_Weekday(train_dataset)
    train_dataset=get_hour_feature(train_dataset,True)
    train_dataset=get_public_holiday(train_dataset)
    return train_dataset

In [6]:
train_dataset=dataset_transformation_operations(train_dataset)
test_dataset=dataset_transformation_operations(test_dataset)

In [7]:
print(train_dataset.head())

   id           date      speed            datetime  month  day  year  hour  \
0   0  1/1/2017 0:00  43.002930 2017-01-01 00:00:00      1    1  2017     0   
1   1  1/1/2017 1:00  46.118696 2017-01-01 01:00:00      1    1  2017     1   
2   2  1/1/2017 2:00  44.294158 2017-01-01 02:00:00      1    1  2017     2   
3   3  1/1/2017 3:00  41.067468 2017-01-01 03:00:00      1    1  2017     3   
4   4  1/1/2017 4:00  46.448653 2017-01-01 04:00:00      1    1  2017     4   

   Sun  Sat  Fri  Thu  Wed  Tue  Mon hour_t  holiday  
0    1    0    0    0    0    0    0      a        0  
1    1    0    0    0    0    0    0      a        0  
2    1    0    0    0    0    0    0      a        0  
3    1    0    0    0    0    0    0      a        0  
4    1    0    0    0    0    0    0      b        0  


In [8]:
def set_timezone(dt):
    cst_tz = timezone('Asia/Shanghai')  
    utc_tz = timezone('Asia/Shanghai') 
    dt["datetime"]=dt["datetime"].apply(lambda x:x.replace(tzinfo=utc_tz).astimezone(cst_tz))
    return dt

def get_merge_result(dt,data_2017,data_2018):
    data_2017.drop_duplicates(subset=['datetime'],keep='first',inplace=True)
    data_2018.drop_duplicates(subset=['datetime'],keep='first',inplace=True)
    result=pd.merge(dt,data_2017,how='left',on=['datetime'])
    result=pd.merge(result,data_2018,how='left',on=['datetime'])
    return result

train_dataset=set_timezone(train_dataset)
print(train_dataset.head())

   id           date      speed                  datetime  month  day  year  \
0   0  1/1/2017 0:00  43.002930 2017-01-01 00:00:00+08:00      1    1  2017   
1   1  1/1/2017 1:00  46.118696 2017-01-01 01:00:00+08:00      1    1  2017   
2   2  1/1/2017 2:00  44.294158 2017-01-01 02:00:00+08:00      1    1  2017   
3   3  1/1/2017 3:00  41.067468 2017-01-01 03:00:00+08:00      1    1  2017   
4   4  1/1/2017 4:00  46.448653 2017-01-01 04:00:00+08:00      1    1  2017   

   hour  Sun  Sat  Fri  Thu  Wed  Tue  Mon hour_t  holiday  
0     0    1    0    0    0    0    0    0      a        0  
1     1    1    0    0    0    0    0    0      a        0  
2     2    1    0    0    0    0    0    0      a        0  
3     3    1    0    0    0    0    0    0      a        0  
4     4    1    0    0    0    0    0    0      b        0  


In [9]:
result=get_merge_result(train_dataset,tmp_data_2017,tmp_data_2018)
print(len(result))
print(result.head())

14006
   id           date      speed                  datetime  month  day  year  \
0   0  1/1/2017 0:00  43.002930 2017-01-01 00:00:00+08:00      1    1  2017   
1   1  1/1/2017 1:00  46.118696 2017-01-01 01:00:00+08:00      1    1  2017   
2   2  1/1/2017 2:00  44.294158 2017-01-01 02:00:00+08:00      1    1  2017   
3   3  1/1/2017 3:00  41.067468 2017-01-01 03:00:00+08:00      1    1  2017   
4   4  1/1/2017 4:00  46.448653 2017-01-01 04:00:00+08:00      1    1  2017   

   hour  Sun  Sat  Fri  Thu  Wed  Tue  Mon hour_t  holiday  Temp_x  Temp_y  
0     0    1    0    0    0    0    0    0      a        0     NaN     NaN  
1     1    1    0    0    0    0    0    0      a        0     NaN     NaN  
2     2    1    0    0    0    0    0    0      a        0     NaN     NaN  
3     3    1    0    0    0    0    0    0      a        0     NaN     NaN  
4     4    1    0    0    0    0    0    0      b        0     NaN     NaN  


In [10]:
print(len(train_dataset))
print(result["Temp_x"])
print(result["Temp_y"])

14006
0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
5          NaN
6          NaN
7          NaN
8        200.0
9        200.0
10       210.0
11       217.0
12       210.0
13       220.0
14       229.0
15       220.0
16       210.0
17       210.0
18       210.0
19       210.0
20       207.0
21       210.0
22       200.0
23       198.0
24       200.0
25       200.0
26       194.0
27       190.0
28       190.0
29       191.0
         ...  
13976      NaN
13977      NaN
13978      NaN
13979      NaN
13980      NaN
13981      NaN
13982      NaN
13983      NaN
13984      NaN
13985      NaN
13986      NaN
13987      NaN
13988      NaN
13989      NaN
13990      NaN
13991      NaN
13992      NaN
13993      NaN
13994      NaN
13995      NaN
13996      NaN
13997      NaN
13998      NaN
13999      NaN
14000      NaN
14001      NaN
14002      NaN
14003      NaN
14004      NaN
14005      NaN
Name: Temp_x, Length: 14006, dtype: float64
0          NaN
1          NaN
2    

In [11]:
a=list(result["Temp_x"])
b=list(result["Temp_y"])

In [12]:
import math
c=[]
for i in range(len(a)):
    if not math.isnan(a[i]):
        c.append(a[i])
    else:
        c.append(b[i])
print(len(c),c[:50],c[-50:])

14006 [nan, nan, nan, nan, nan, nan, nan, nan, 200.0, 200.0, 210.0, 217.0, 210.0, 220.0, 229.0, 220.0, 210.0, 210.0, 210.0, 210.0, 207.0, 210.0, 200.0, 198.0, 200.0, 200.0, 194.0, 190.0, 190.0, 191.0, 190.0, 190.0, 197.0, 210.0, 220.0, 222.0, 230.0, 230.0, 238.0, 240.0, 230.0, 235.0, 220.0, 220.0, 209.0, 210.0, 210.0, 207.0, 210.0, 200.0] [170.0, 170.0, 190.0, 192.0, 190.0, 205.0, 200.0, 180.0, 180.0, 172.0, 170.0, 161.0, 160.0, 150.0, 142.0, 140.0, 132.0, 130.0, 126.0, 130.0, 138.0, 140.0, 140.0, 129.0, 130.0, 122.0, 117.0, 113.0, 95.0, 110.0, 113.0, 120.0, 130.0, 130.0, 120.0, 120.0, 120.0, 110.0, 116.0, 110.0, 110.0, 100.0, 103.0, 110.0, 110.0, 120.0, 120.0, 120.0, 120.0, 116.0]


In [13]:
for i in range(8):
    c[i]=190.0
print(c)

[190.0, 190.0, 190.0, 190.0, 190.0, 190.0, 190.0, 190.0, 200.0, 200.0, 210.0, 217.0, 210.0, 220.0, 229.0, 220.0, 210.0, 210.0, 210.0, 210.0, 207.0, 210.0, 200.0, 198.0, 200.0, 200.0, 194.0, 190.0, 190.0, 191.0, 190.0, 190.0, 197.0, 210.0, 220.0, 222.0, 230.0, 230.0, 238.0, 240.0, 230.0, 235.0, 220.0, 220.0, 209.0, 210.0, 210.0, 207.0, 210.0, 200.0, 193.0, 190.0, 190.0, 193.0, 190.0, 190.0, 202.0, 220.0, 230.0, 244.0, 240.0, 240.0, 240.0, 240.0, 240.0, 232.0, 220.0, 210.0, 213.0, 210.0, 210.0, 206.0, 200.0, 200.0, 197.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 210.0, 220.0, 232.0, 240.0, 250.0, 252.0, 260.0, 250.0, 239.0, 240.0, 230.0, 221.0, 220.0, 220.0, 215.0, 210.0, 200.0, 213.0, 210.0, 210.0, 205.0, 210.0, 210.0, 210.0, 220.0, 240.0, 245.0, 230.0, 230.0, 230.0, 230.0, 240.0, 234.0, 230.0, 230.0, 225.0, 230.0, 230.0, 221.0, 220.0, 220.0, 221.0, 220.0, 220.0, 214.0, 200.0, 200.0, 203.0, 220.0, 220.0, 227.0, 230.0, 250.0, 252.0, 250.0, 250.0, 250.0, 240.0, 240.0, 228.0, 220.0, 220.0

In [14]:
result["Temp"]=c

In [15]:
print(result)

          id              date      speed                  datetime  month  \
0          0     1/1/2017 0:00  43.002930 2017-01-01 00:00:00+08:00      1   
1          1     1/1/2017 1:00  46.118696 2017-01-01 01:00:00+08:00      1   
2          2     1/1/2017 2:00  44.294158 2017-01-01 02:00:00+08:00      1   
3          3     1/1/2017 3:00  41.067468 2017-01-01 03:00:00+08:00      1   
4          4     1/1/2017 4:00  46.448653 2017-01-01 04:00:00+08:00      1   
5          5     1/1/2017 5:00  46.797766 2017-01-01 05:00:00+08:00      1   
6          6     1/1/2017 6:00  44.404925 2017-01-01 06:00:00+08:00      1   
7          7     1/1/2017 7:00  45.255897 2017-01-01 07:00:00+08:00      1   
8          8     1/1/2017 8:00  45.680859 2017-01-01 08:00:00+08:00      1   
9          9     1/1/2017 9:00  48.435676 2017-01-01 09:00:00+08:00      1   
10        10    1/1/2017 10:00  49.457054 2017-01-01 10:00:00+08:00      1   
11        11    1/1/2017 11:00  41.968519 2017-01-01 11:00:00+08

[14006 rows x 20 columns]


In [16]:
category_vars=['hour_t','hour','month','day']
number_vars=["Temp",'year','Sat','Sun','Mon','Fri','Thu','Wed','Tue','holiday']

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelBinarizer,OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

n_transformation=Pipeline(steps=[('scaler', StandardScaler())])
c_transformation=Pipeline(steps=[('oneHot',OneHotEncoder())])

preprocessor=ColumnTransformer(transformers=[('num',n_transformation,number_vars),
    ('cat',c_transformation,category_vars)])

train_dataset_m=preprocessor.fit_transform(result)

In [18]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(train_dataset_m,train_dataset['speed'],test_size=0.15,random_state=42)

In [19]:
print(X_train.shape)

(11905, 84)


In [20]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error as MSE

model = XGBRegressor()

param_grid = {'nthread':[4], 
              'objective':['reg:squarederror'],
              'learning_rate':[.1, .03, .05],
              'max_depth':[8],
              'min_child_weight':[5],
              'subsample':[0.7],
              'colsample_bytree':[0.7],
              #'gamma':[0.1,0.15],
              'n_estimators':[500]}

GS=GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=2)
GS.fit(X_train,y_train)
y_pred=GS.predict(X_test)
print(MSE(y_pred,y_test))

D:\Anaconda\Ana\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


10.933115159363178


In [21]:
test_dataset=set_timezone(test_dataset)
test_result=get_merge_result(test_dataset,tmp_data_2017,tmp_data_2018)
a=list(test_result["Temp_x"])
b=list(test_result["Temp_y"])
import math
c=[]
for i in range(len(a)):
    if not math.isnan(a[i]):
        c.append(a[i])
    else:
        c.append(b[i])
        
test_result["Temp"]=c

test_dataset_m=preprocessor.fit_transform(test_result)
y_pred_test=GS.predict(test_dataset_m)
y_p_modify=list(y_pred_test)
rs=[]
for i in y_p_modify:
    rs.append(i*1)
pred=pd.DataFrame({'speed':rs},columns=['speed'])
result=pd.concat([test_data.drop('date',axis=1),pred],axis=1)
result.to_csv('E:\\5001\\KAGGLE\\Result_BOGUS_26.csv',index=False)